In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
tf.test.gpu_device_name

<function tensorflow.python.framework.test_util.gpu_device_name()>

In [3]:
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/Tomato_train.zip"
with ZipFile(file_name, "r") as zip:
  zip.extractall()
  print("Done")

Done


In [4]:
from zipfile import ZipFile
file_name_test = "/content/drive/MyDrive/tomato_test.zip"
with ZipFile(file_name_test, "r") as zip:
  zip.extractall()
  print("Done")

Done


In [5]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [6]:

import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:

train_path = os.path.join("/content/Tomato")
print(os.listdir(train_path))
print("*"*100)
valid_path = os.path.join("/content/tomato_test/tom")
print(os.listdir(valid_path))

['Tomato___Early_blight', 'Tomato___Septoria_leaf_spot', 'Tomato___Bacterial_spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Late_blight', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___healthy', 'Tomato___Leaf_Mold']
****************************************************************************************************
['Tomato___Early_blight', 'Tomato___Septoria_leaf_spot', 'Tomato___Bacterial_spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Late_blight', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___healthy', 'Tomato___Leaf_Mold']


In [8]:
from glob import glob
folders = glob("/content/Tomato/*")
len(folders)

10

In [9]:
# re-size all the images to this

batch_size = 32
img_height = 224
img_width = 224
img_size = (img_height, img_width)
img_size

(224, 224)

In [10]:

train_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=0.2, 
  subset="training",
  seed=123,
  image_size=(img_height, img_width),batch_size=batch_size)

Found 18345 files belonging to 10 classes.
Using 14676 files for training.


In [11]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  valid_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4585 files belonging to 10 classes.
Using 917 files for validation.


In [12]:
class_names = train_ds.class_names
print(class_names )

['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 16))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 224, 224, 3)
(32,)


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

0.0 0.8121078


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Building the Model

In [ ]:
IMAGE_SIZE = [224,224]
IMAGE_SIZE

[224, 224]

In [ ]:
# Import the InceptionV3 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
vgg19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 4s 0us/step


In [ ]:
# don't train existing weights
for layer in vgg19.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/Tomato/*')

In [ ]:
x = Flatten()(vgg19.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

model_VGG19 = Model(inputs=vgg19.input, outputs=prediction)

In [ ]:
# view the structure of the model

model_VGG19.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model_VGG19.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)

valid_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/Tomato',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
     

Found 18345 images belonging to 10 classes.


In [ ]:
valid_set = valid_datagen.flow_from_directory('/content/tomato_test/tom',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 4585 images belonging to 10 classes.


In [ ]:

model_VGG19.fit(training_set,
                       validation_data=valid_set,
                       validation_batch_size=64,
                       epochs=15)


Epoch 1/15
574/574 [==============================] - 142s 231ms/step - loss: 0.7285 - accuracy: 0.7617 - val_loss: 0.4603 - val_accuracy: 0.8473
Epoch 2/15
574/574 [==============================] - 133s 231ms/step - loss: 0.3183 - accuracy: 0.8933 - val_loss: 0.3341 - val_accuracy: 0.8879
Epoch 3/15
574/574 [==============================] - 133s 231ms/step - loss: 0.1895 - accuracy: 0.9350 - val_loss: 0.3594 - val_accuracy: 0.8785
Epoch 4/15
574/574 [==============================] - 132s 231ms/step - loss: 0.1179 - accuracy: 0.9615 - val_loss: 0.3723 - val_accuracy: 0.8811
Epoch 5/15
574/574 [==============================] - 132s 231ms/step - loss: 0.1071 - accuracy: 0.9656 - val_loss: 0.3041 - val_accuracy: 0.9003
Epoch 6/15
574/574 [==============================] - 133s 231ms/step - loss: 0.0792 - accuracy: 0.9745 - val_loss: 0.3620 - val_accuracy: 0.8907
Epoch 7/15
574/574 [==============================] - 132s 231ms/step - loss: 0.0787 - accuracy: 0.9732 - val_loss: 0.5500 -

In [ ]:
model = model_VGG19.save('/content/model_VGG19.h5')

In [ ]:

score_train = model_VGG19.evaluate(training_set, verbose=2)
accuracy_train = 100*score_train[1]
print('Train accuracy = %.4f%% ' % accuracy_train) 

score_val = model_VGG19.evaluate(valid_set, verbose=2)
accuracy_val = 100*score_val[1]
print('Validate accuracy = %.4f%% ' % accuracy_val)

574/574 - 94s - loss: 0.0499 - accuracy: 0.9803 - 94s/epoch - 163ms/step
Train accuracy = 98.0267% 
144/144 - 23s - loss: 0.5178 - accuracy: 0.8862 - 23s/epoch - 163ms/step
Validate accuracy = 88.6150% 


In [ ]:
import tensorflow as tf
import os 

In [ ]:
# convert keras model to tflite 
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [ ]:
from keras.models import load_model
model = load_model("/content/model_VGG19.h5")

In [ ]:
TF_LITE_MODEL_FILE_NAME = "tflite_model.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# or using another method

# Save the model.
with open('tflite_model_another.tflite', 'wb') as f:
  f.write(tflite_model)

File size: 79210.273 Kilobytes


In [ ]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [  1 224 224   3]
Input Type: <class 'numpy.float32'>
Output Shape: [ 1 10]
Output Type: <class 'numpy.float32'>
